In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
rml_df = pd.read_csv('./data/rml_df.csv')

In [3]:
rml_df.sort_values(by=['patient_id' , 'stage_start_time'], inplace=True)
rml_df.reset_index(drop=True, inplace=True)
rml_df = rml_df[rml_df['patient_id'] != 1339]

In [4]:
sleep_df = rml_df[['patient_id', 'stage_start_time', 'stage_end_time', 'stage_duration', 'stage_type']].copy()

In [5]:
sleep_df.drop_duplicates(inplace=True)

In [6]:
sleep_df = sleep_df[sleep_df['patient_id'] != 1339]

In [7]:
sleep_df.reset_index(drop=True, inplace=True)

In [8]:
sleep_df

,patient_id,stage_start_time,stage_end_time,stage_duration,stage_type
0,995,0.0,210.0,210.0,Wake
1,995,210.0,300.0,90.0,NonREM1
2,995,300.0,870.0,570.0,NonREM2
3,995,870.0,930.0,60.0,Wake
4,995,930.0,960.0,30.0,NonREM1
...,...,...,...,...,...
14078,1701,17250.0,17280.0,30.0,NonREM1
14079,1701,17280.0,17700.0,420.0,NonREM2
14080,1701,17700.0,17820.0,120.0,NonREM1
14081,1701,17820.0,18840.0,1020.0,NonREM2


In [9]:
save_dir = 'E:/REM Detection/data/extracted/wav'

In [ ]:
def get_file_name_list(path):
    file_name_list = []
    for root, dirs, files in os.walk(path):
        for file in files:
            file_name_list.append(file.split('.')[0])
    return file_name_list

file_name_list = get_file_name_list(save_dir)
file_name_list

In [11]:
file_name_list.sort()

In [12]:
len(file_name_list)

14083

In [13]:
audio_sleep_df = pd.concat([sleep_df, pd.DataFrame(file_name_list, columns=['raw_audio_file_id'])], axis=1)

In [14]:
audio_sleep_df

,patient_id,stage_start_time,stage_end_time,stage_duration,stage_type,raw_audio_file_id
0,995,0.0,210.0,210.0,Wake,00000995_01_00
1,995,210.0,300.0,90.0,NonREM1,00000995_01_01
2,995,300.0,870.0,570.0,NonREM2,00000995_01_02
3,995,870.0,930.0,60.0,Wake,00000995_01_03
4,995,930.0,960.0,30.0,NonREM1,00000995_01_04
...,...,...,...,...,...,...
14078,1701,17250.0,17280.0,30.0,NonREM1,00001701_05_29
14079,1701,17280.0,17700.0,420.0,NonREM2,00001701_05_30
14080,1701,17700.0,17820.0,120.0,NonREM1,00001701_05_31
14081,1701,17820.0,18840.0,1020.0,NonREM2,00001701_05_32


In [15]:
rml_df

,patient_id,stage_start_time,stage_end_time,stage_duration,stage_type,event_start_time,event_end_time,event_duration,respiratory_type
0,995,0.0,210.0,210.0,Wake,NaN,NaN,NaN,NaN
1,995,210.0,300.0,90.0,NonREM1,NaN,NaN,NaN,NaN
2,995,300.0,870.0,570.0,NonREM2,NaN,NaN,NaN,NaN
3,995,870.0,930.0,60.0,Wake,NaN,NaN,NaN,NaN
4,995,930.0,960.0,30.0,NonREM1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
75591,1701,17820.0,18840.0,1020.0,NonREM2,18696.5,18720.0,23.5,Hypopnea
75592,1701,17820.0,18840.0,1020.0,NonREM2,18744.5,18760.5,16.0,Hypopnea
75593,1701,17820.0,18840.0,1020.0,NonREM2,18780.5,18794.0,13.5,Hypopnea
75594,1701,17820.0,18840.0,1020.0,NonREM2,18817.0,18835.0,18.0,Hypopnea


In [16]:
# add the raw_audio_file_id to the rml_df

rml_df = pd.merge(rml_df, audio_sleep_df[['patient_id', 'stage_start_time', 'raw_audio_file_id']], on=['patient_id', 'stage_start_time'], how='left')

In [22]:
# rml_df.to_csv('./data/sleep_stage_respiratory_info.csv', index=False)

In [23]:
# EDA on the rml_df

rml_df['stage_type'].value_counts()

NonREM2      60525
NonREM1       6213
REM           3836
Wake          3399
NonREM3       1531
NotScored        1
Name: stage_type, dtype: int64

In [26]:
# stage type distribution percentage

rml_df['stage_type'].value_counts(normalize=True) * 100

NonREM2      80.160254
NonREM1       8.228594
REM           5.080458
Wake          4.501689
NonREM3       2.027680
NotScored     0.001324
Name: stage_type, dtype: float64